# Introduction: Deploy Anyscale Ray on GCP Compute Engine Instances (GCE)

© 2025, Anyscale. All Rights Reserved

This notebook shows how to create the infrastructure and Anyscale-specific resources needed for GCP Compute Engine (GCE) deployments using the custom **`anyscale cloud register`** method.

Use it as a starting point and replace all placeholders (e.g.&nbsp;`{ANYSCALE_CLOUD_NAME}`) with values from your environment.

**What this creates:**
- VPC and subnet for Anyscale resources
- IAM Service Account for Compute Engine nodes
- GCS Storage Bucket for Anyscale logs and shared resources  
- Filestore instance for shared storage (optional)

**Prerequisites:**
- Proper GCP permissions and authentication


## Prerequisites

Before we begin, ensure you have the following tools installed:

```bash
# Create Google Cloud Project
# https://cloud.google.com/resource-manager/docs/creating-managing-projects


# Install Google Cloud SDK/CLI
# https://cloud.google.com/sdk/docs/install

# Configure Google Cloud CLI Authentication
# https://cloud.google.com/docs/authentication/gcloud

# Install Terraform (version 1.9+)
# https://developer.hashicorp.com/terraform/install

# Install Anyscale CLI (version 0.5.86+)
# https://docs.anyscale.com/reference/quickstart-cli/
```

You also need:
- A GCP Project with billing enabled
- Project Owner or Editor role
- Required APIs enabled (we'll do this in the next step)

## 1. Installation


### 1.1 Configure Google Cloud Authentication

First, set up authentication and configure your GCP project:


In [ ]:
# Set up application default credentials
# This will open a browser window for authentication
!gcloud auth application-default login

# Set your project (replace YOUR_PROJECT_ID with your actual project ID)
# Example: !gcloud config set project my-anyscale-project-123456
!gcloud config set project {GCP_PROJECT_ID}

# Set your region like us-west2
!gcloud config set compute/region {GCP_REGION}

# Verify authentication and configuration
!gcloud auth list
!gcloud config list


### 1.2 Enable Required APIs

Enable all the necessary Google Cloud APIs for this deployment:


In [ ]:
!gcloud services enable \
  compute.googleapis.com \
  storage.googleapis.com \
  file.googleapis.com \
  iam.googleapis.com \
  cloudresourcemanager.googleapis.com

# Verify APIs are enabled
!gcloud services list --enabled --filter="name:compute.googleapis.com OR name:storage.googleapis.com"


## 2. Create Anyscale Resources with Terraform

Let's set up the necessary Terraform variables and apply the configuration:

### Creating Anyscale Resources

This deployment uses the official **Anyscale Terraform modules** from [terraform-google-anyscale-cloudfoundation-modules](https://github.com/anyscale/terraform-google-anyscale-cloudfoundation-modules). These modules provide:

- **Pre-tested configurations** for GCP resources
- **Security best practices** built-in
- **Simplified resource management** with fewer variables to configure
- **Automatic generation** of Anyscale registration commands

Steps for deploying Anyscale resources via Terraform:

* Review and modify [variables.tf](variables.tf) with your own configurations, and (optionally) create a `terraform.tfvars` file to override any of the defaults.
* You can view [main.tf](main.tf) to see how the official Anyscale modules are used.


### 2.1 Create terraform.tfvars

Create a `terraform.tfvars` file with your specific configuration:


Create terraform.tfvars file

Required variables

```
google_project_id = {GCP_PROJECT_ID} # the name of your google project, not the project_number
google_region = {GCP_REGION} # the region of your google project

# Required variables
anyscale_cloud_id = {ANYSCALE_CLOUD_NAME} # the anyscale cloud name you want to use
common_prefix = {PREFIX_FOR_THE_RESOURCES_ADDED} # like: "anyscale-compute-engine"
aws_role_name     = "gcp_if_{YOUR_ANYSCALE_ORG_ID}" # You can get it from your Anyscale account's "Organization Settings", it is a string like "org_xxxxxxx". Add this value with prefix "gcp_if_"

# Optional variables
# bucket_name = "my-custom-bucket-name"  # Uncomment to use custom bucket name
# enable_filestore = true
# filestore_capacity_gb = 1024
# filestore_tier = "STANDARD"
```

Note: The VPC will be created automatically by Terraform using the configuration in `vpc.tf`.

### 2.2 Run Terraform Commands

Now let's initialize and deploy the Terraform configuration:


In [ ]:
# Initialize Terraform
!terraform init

# Preview the changes
!terraform plan

# Apply the changes (this may take 10-15 minutes)
!terraform apply -auto-approve


<div class="alert alert-block alert-info">
<b>Take a note to the output of terraform apply! </b>You will need it when you register the Anyscale cloud to your cloud provider.
</div>

<details>
<summary>Sample output</summary>
```
Outputs:

anyscale_registration_command = <<EOT
anyscale cloud register \
    --name <anyscale_cloud_name> \
    --provider gcp \
    --region us-west2 \
    --compute-stack vm \
    --anyscale-operator-iam-identity anyscale-nodes-d3a9a7d0@xing-compute-engine-test.iam.gserviceaccount.com \
    --cloud-storage-bucket-name anyscale-compute-engine-bucket-xxx \
    --project-id xxx \
    --vpc-name xxx \
    --subnet-names anyscale-subnet \
    --anyscale-service-account-email anyscale-nodes-xxx@xxx.iam.gserviceaccount.com \
    --instance-service-account-email anyscale-nodes-xxx@xxx.iam.gserviceaccount.com \
    --provider-name projects/xxx/locations/global/workloadIdentityPools/xxxx/providers/anyscale-provider \
    --firewall-policy-names anyscale-firewall-policy-xxxx \
    --file-storage-id anyscale-filestore-xxxx \
    --filestore-location xxxxx-a
EOT
compute_nodes_service_account_email =
filestore_instance_name = 
filestore_location = 
firewall_policy_name = 
gcs_bucket_name = 
workload_identity_pool_provider = 
```
</details>

## 3. Register the Anyscale Cloud

First, ensure you're logged into Anyscale. if you cannot run it in console, please run it in your local terminal:


In [ ]:
# First, ensure you're logged into Anyscale
!anyscale login

Then use the output of `terraform apply` to register Anyscale cloud. You only need to replace the `--name` parameter with your preferred `ANYSCALE_CLOUD_NAME`. The command looks like:

```bash
anyscale cloud register ...
```

You will get output like:

```text
Output
(anyscale +18.0s) Successfully created cloud anyscale-cloud-gce-private-xxxxx, and it's ready to use.
```

## 4. Test

Once the cloud is registered, you cOnce the cluster is created, you can test it by submitting a job:an test it by submitting a job:


In [ ]:
A_RANDOM_JOB_NAME = "A random job name"  # Replace with a random job name you like

!cd ../test && python test_job.py --cloud-name {ANYSCALE_CLOUD_NAME} --stack-type vm --cloud-provider gcp

# You can check the job status by running:
!anyscale job list --cloud {ANYSCALE_CLOUD_NAME}

You just start a job and you can see the logs from your Anyscale Console. You can view the running results from Anyscale console in "Jobs".

## 5. Cleanup

When you're done, you can terminate the cluster and clean up resources:


In [ ]:
# Step 1: Delete any running clusters
!anyscale cluster terminate --name gcp-compute-ray-cluster-autoscaled

# Step 2: Delete Anyscale Cloud Registration
!anyscale cloud delete {ANYSCALE_CLOUD_NAME}

# Step 3: Empty GCS bucket (if it has objects)
# (Optional) If you want to delete the GCS bucket, you can run the following command:
# Find your GCS bucket name from the output of the terraform apply command, then run the following command:
!gsutil rm -r gs://{GCS_BUCKET_NAME}/*


# Step 4: Destroy Terraform resources
!terraform destroy -auto-approve

# Optional: # You may need to delete firewall policy created by terraform manually.